In [33]:
import numpy as np
import dxchange

from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib inline
import tkinter as tk
from tkinter.filedialog import askopenfile

In [34]:
# Utility Functions

In [35]:
def read_filename():
    window = tk.Tk()
    window.wm_attributes('-topmost', 1)
    window.withdraw()

    file = askopenfile(mode ='r', filetypes =[('Python Files', '*.h5')])
    window.destroy()
    return file.name

# Load data

In [36]:
# !wget -nc https://anl.box.com/shared/static/or8vlzdu07d8zwxvk50ihwghq39ide0o.npz
file_name = read_filename()

2024-01-25 14:52:07.744 python[98781:7726538] +[CATransaction synchronize] called within transaction
2024-01-25 14:52:07.811 python[98781:7726538] +[CATransaction synchronize] called within transaction
2024-01-25 14:52:09.705 python[98781:7726538] +[CATransaction synchronize] called within transaction


# Read data, dark and flat fields, projection angles, the rotation center and metadata

In [37]:
# data = np.random.rand(100,256,256) #3d-array with 100 frames 256x256
data, flat, dark, theta = dxchange.read_aps_tomoscan_hdf5(file_name)#, sino=(100, 400))
meta_dict = dxchange.read_hdf_meta(file_name)

In [38]:
data.shape 

(1800, 852, 2800)

# Plot projections

In [40]:
def plot_projections(angle=1):
    plt.imshow(data[angle,:],cmap='gray')
interact(plot_projections, angle = widgets.IntSlider(name='Projection', value=data.shape[0]//2,
                                               min=0,
                                               max=data.shape[0]-1,
                                               step=1))

interactive(children=(IntSlider(value=900, description='angle', max=1799), Output()), _dom_classes=('widget-in…

<function __main__.plot_projections(angle=1)>